In [ ]:
from typing import Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import (
    FewShotPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

In [ ]:
# b = chat.predict("How many planets are there?")
# b

template = PromptTemplate.from_template(
    "{country_a}와 {country_b}사이의 거리는 어떻게되나요?"
)
prompt = template.format(country_a="맥시코", country_b="인도")
chat.predict(prompt)

In [ ]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 지리 전문가입니다. 그리고 당신은 {language}로만 대답을 합니다.",
        ),
        ("ai", "안녕하세요 제 이름은 {name}입니다."),
        (
            "human",
            "{country_a}와 {country_b}사이의 거리는 어떻게되나요? 그리고 당신의 이름은 무었입니까?",
        ),
    ]
)
prompt = template.format_messages(
    language="일본어", name="길동", country_a="일본", country_b="한국"
)
chat.predict_messages(prompt)

In [ ]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))


p = CommaOutputParser()
p.parse(" a ,  b ,  c ,   d")

In [ ]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do Not reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)
# prompt = template.format_messages(
#     max_items=10,
#     question="What are the planets"
# )
prompt = template.format_messages(max_items=10, question="What are the colors")
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)

In [ ]:
chain = template | chat | CommaOutputParser()
chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

In [ ]:
chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class international chef. You create easy to follow recipies for any type of cuisins with easy to find ingredients.",
        ),
        ("human", "I want to cook {cuisine} food"),
    ]
)
chef_chain = chef_prompt | chat
# chef_chain.invoke({
#     "cuisine":"korean"
# })

In [ ]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find a alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)

veg_chain = veg_chef_prompt | chat

In [ ]:
final_chain = {"recipe": chef_chain} | veg_chain
final_chain.invoke({"cuisine": "korean"})

In [ ]:
# 4.1 FewShotPromptTemplate

# t = PromptTemplate.from_template("What is the capital of {country}")
# t.format(country="Korea")

# t=PromptTemplate(template="What is the capital of {country}", input_variables=["country"])
# t.format(country="japan")

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
    },
]

example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

chain = prompt | chat
chain.invoke({"country": "Germany"})

In [ ]:
# 4.2 FewShotChatMessagePromptTemplate
examples = [
    {
        "country": "France",
        "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
    },
    {
        "country": "Italy",
        "answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
    },
    {
        "country": "Greece",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give shorts answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat
chain.invoke({"country": "Korea"})

In [ ]:
# 4.3 LengthBasedExampleSelector

from typing import Any, Dict


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]

    def add_example(self, example):
        self.examples.append(example)


example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

# example_selector = LengthBasedExampleSelector(
#     examples=examples,
#     example_prompt=example_prompt,
#     max_length=10,
# )
example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)
prompt.format(country="Korea")

# chain = prompt | chat
# chain.invoke({"country": "Germany"})

In [ ]:
# 4.4 Serialization and Composition
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.yaml")

prompt.format(country="Germany")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)
example = PromptTemplate.from_template(
    """
    This is and example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)
start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You: 
"""
)
final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]
full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

# full_prompt.format(
#     character="Pirate",
#     example_question="What is your location?",
#     example_answer="Arrrrg! That is a secret!! Arg arg!!",
#     question="What is your fav food?",
# )

chain = full_prompt | chat
chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

In [ ]:
# 4.5 Chaching
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)
chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler()
    # ]
)
chat.predict("How do you make italian pasta?")

In [ ]:
# 4.6 Usage

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju?")
    b = chat.predict("What is the recipe for bread?")
    print(a, b, "\n")
    print(usage)

In [ ]:
# 4.6 Serialization
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

# chat = OpenAI(
# temperature=0.1,
# max_tokens=450,
# model="gpt-3.5-turbo-16k"
# )
# chat.save("model.json")

chat = load_llm("model.json")
print(chat)

In [ ]:
# 5.0 ConversationBufferMemorry
# 단점 - 대화 내용이 길어질수록 메모리도 계속 커지니깐 비효율적
# 모델에게 매번 보내야될 대화기록이 길어짐 - 비용증가
# 장점 - text completion할때 유용
# 예측할때, 텍스트를 자동완성활때
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "Hi"}, {"output": "How are you?"})
memory.load_memory_variables({})

In [ ]:
# 5.1 ConversationBufferWindowMemory

# 장점 - 메모리 사이즈가 계속 늘어나지 않는다.
# 단점 - 챗봇이 예전 대화를 기억하기 힘들다.
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(return_messages=True, k=4)


def add_chat(input, output):
    memory.save_context({"input": input}, {"output": output})


add_chat(1, 1)
add_chat(2, 2)
add_chat(3, 3)
add_chat(4, 4)
add_chat(5, 5)
add_chat(6, 6)
memory.load_memory_variables({})

In [ ]:
# 5.2 ConversationSummaryMemory

from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Anderson, I live in South Korea", "Wow that is so cool!!")
add_message("South Korea is so pretty", "I wish I could go!!!")
get_history()

In [ ]:
# 5.3 ConversationSummaryBufferMemory

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.globals import set_debug

set_debug(False)
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=10,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi", "How are you?")
add_message("I'm Fine ", "How are you?")

get_history()

In [ ]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm Anderson, I live in South Korea", "Wow that is so cool!")
memory.load_memory_variables({"inputs": "who is Anderson"})

In [ ]:
add_message("Anderson likes kimchi", "Wow that is so cool!")

In [ ]:
memory.load_memory_variables({"inputs": "What does anderson like"})

In [ ]:
# 5.5 Memory on LLMChain

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
)
template = """
    You are a helpful AI talking to a human

    {chat_history}
    Human:{question}
    You:
"""
chain = LLMChain(
    llm=llm, memory=memory, prompt=PromptTemplate.from_template(template), verbose=True
)

chain.predict(question="My name is Anderson")

In [ ]:
chain.predict(question="I live in Seoul")

In [ ]:
chain.predict(question="What is my name?")

In [ ]:
# 5.6 Chat Based Memory  ===> 오류발생 해결안됨
# => 수정완료 => return_message 철자 오류 => return_messages

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(llm=llm, memory=memory, prompt=prompt, verbose=True)

chain.predict(question="My name is Anderson")

In [ ]:
# 5.7 LCEL Based Memory => 이것도 오류 발생함......

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    # memory_key="chat_history"  # default key 이므로 삭제 가능
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(input):
    print(input)
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )

    print(result)


invoke_chain("My name is Anderson")

In [ ]:
invoke_chain("I live in Seoul")

In [ ]:
invoke_chain("What is my name")

In [ ]:
# 6 RAG (Retrieval-Augmented Generation) 검색-증강 생성
# 6.1 Data Loaders and Splitters

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
)


# llm = ChatOpenAI(temperature=0.1)

# splitter = RecursiveCharacterTextSplitter(
#     chunk_size=200,
#     chunk_overlap=50
# )
splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.docx")
loader.load_and_split(text_splitter=splitter)

In [ ]:
# 6 RAG (Retrieval-Augmented Generation) 검색-증강 생성
# 6.4 Vector Store

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/")
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.docx")
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cached_embeddir = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = Chroma.from_documents(docs, cached_embeddir)

In [ ]:
# 6 RAG (Retrieval-Augmented Generation) 검색-증강 생성
# 6.6 RetrievalQA

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()
cache_dir = LocalFileStore("./.cache/")
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.docx")
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cached_embeddir = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddir)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",  # "map_reduce", #"refine", #"stuff",
    retriever=vectorstore.as_retriever(),
)

# chain.run("Where does Winston live?")
chain.run("Describe Victory Mansions")

In [ ]:
# 6 RAG (Retrieval-Augmented Generation) 검색-증강 생성
# 6.8 Stuff LCEL Chain

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)
cache_dir = LocalFileStore("./.cache/")
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.docx")
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cached_embeddir = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddir)

# chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="map_rerank", #"map_reduce", #"refine", #"stuff",
#     retriever=vectorstore.as_retriever(),
# )
retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)
chain = {"context": retriver, "question": RunnablePassthrough()} | prompt | llm
# chain.run("Where does Winston live?")
chain.invoke("Describe Victory Mansions")

In [ ]:
# 6 RAG (Retrieval-Augmented Generation) 검색-증강 생성
# 6.9 Map Reduce LCEL Chain

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(temperature=0.1)
cache_dir = LocalFileStore("./.cache/")
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.docx")
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cached_embeddir = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddir)

retriever = vectorstore.as_retriever()

map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    # print(inputs)
    documents = inputs["documents"]
    question = inputs["question"]

    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        for doc in documents
    )


map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
    Given the following extracted parts of a long document and a question, create a final answer.",
    "If you don't know the answer, just say that you don't know. Don't try to make up an answer.",
    ------
     {context}
    """,
        ),
        ("human", "{question}"),
    ]
)


chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm
# chain.invoke("Describe Victory Mansions")
chain.invoke("Where does Winston go to work?")

In [ ]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("A {word} is a")

llm = HuggingFacePipeline.from_model_id(
    model_id="openai-community/gpt2",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 50},
)

chain = prompt | llm
chain.invoke({"word": "potato"})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate


def get_weather(lon, lat):
    print("call function...")


function = {
    "name": "get_weather",
    "description": "function that takes longitude and latitude to find the weather of a place",
    "parameters": {
        "type": "object",
        "properties": {
            "lon": {"type": "string", "description": "The longitude coordinate"},
            "lan": {"type": "string", "description": "The latitude coordinate"},
        },
    },
    "required": ["lon", "lat"],
}
llm = ChatOpenAI(temperature=0.1).bind(function_call="auto", functions=[function])

prompt = PromptTemplate.from_template("Who is the weather in {city}?")

chain = prompt | llm
response = chain.invoke({"city": "korea"})
args = response.additional_kwargs["function_call"]["arguments"]

In [ ]:
import json

r = json.loads(args)
get_weather(r["lon"], r["lan"])

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate


function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}

llm = ChatOpenAI(temperature=0.1).bind(
    function_call={"name": "create_quiz"},
    functions=[function],
)

prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm
response = chain.invoke({"city": "korea"})
args = response.additional_kwargs["function_call"]["arguments"]

In [ ]:
print(args)

In [ ]:
import subprocess
from pydub import AudioSegment
import math


def extract_audio_from_video(video_path, audio_path):
    command = ["ffmpeg", "-i", video_path, "-vn", audio_path]
    subprocess.run(command)


def cut_audio_in_chunks(audio_path, minutes, dest_path):
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = minutes * 60 * 1000
    chunks = math.ceil(len(track) / chunk_len)
    for i in range(chunks):
        start_time = i * chunk_len
        end_time = (i + 1) * chunk_len
        chunk = track[start_time:end_time]
        chunk.export(f"{dest_path}/chunk_{i}.mp3", format="mp3")

In [ ]:
# extract_audio_from_video("./files/test2.mp4", "./files/test2.mp3")
cut_audio_in_chunks("./files/test2.mp3", 10, "./files/chunks")

In [ ]:
import openai
import glob


def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(f"{chunk_folder}/*.mp3")
    final_transcript = ""
    for file in files:
        with open(file, "rb") as audio_file, open(destination, "a") as dest_file:
            transcript = openai.Audio.transcribe(
                "whisper-1",
                audio_file,
            )
            dest_file.write(transcript["text"])


transcribe_chunks("./files/chunks", "./files/transcript.txt")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def plus(a, b):
    return a + b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[
        StructuredTool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. This tool take two arguments, both should be numbers",
        )
    ],
)
prompt = "Cost of $355.23 + $234.23 + $123.23 + $232.23"

agent.invoke(prompt)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool, Tool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def plus(inputs):
    a, b = inputs.split(",")
    return float(a) + float(b)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    tools=[
        Tool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. Use this tool by sending a pair of number separated a common.\nExample:1,2",
        )
    ],
)
prompt = "Cost of $355.23 + $234.23 + $123.23 + $232.23"

agent.invoke(prompt)

In [ ]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The first number")
    b: float = Field(description="The second number")


class CalculatorTool(BaseTool):
    name = "SumsCalculator"
    description = """
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
    args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return a + b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[CalculatorTool()],
)
prompt = "Cost of $355.23 + $234.23 + $123.23 + $232.23"

agent.invoke(prompt)

In [ ]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1)


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will serach for")


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    Example query: Stock Market Symbol for Apple Company
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = (
        StockMarketSymbolSearchToolArgsSchema
    )

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[StockMarketSymbolSearchTool()],
)
prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have. in korean"

agent.invoke(prompt)

In [ ]:
import os
import requests
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

llm = ChatOpenAI(temperature=0.1)


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will serach for")


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    Example query: Stock Market Symbol for Apple Company
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = (
        StockMarketSymbolSearchToolArgsSchema
    )

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverviewToolArgsSchema(BaseModel):
    symbol: str = Field(description="Stock symbol of the company.Example: APPL, TSLA")


class CompanyOverviewTool(BaseTool):
    name = "CompanyOverviewTool"
    description = """
    Use this to get an iverview of the financials of the company.
    You should enter a stock symbol
    """
    args_schema: Type[CompanyOverviewToolArgsSchema] = CompanyOverviewToolArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatementTool"
    description = """
    Use this to get the income statement of a company.
    You should enter a stock symbol
    """
    args_schema: Type[CompanyOverviewToolArgsSchema] = CompanyOverviewToolArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        response = r.json()
        return list(response["Weekly Time Series"].items())[:200]


class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformanceTool"
    description = """
    Use this to get the weekly performance of a company.
    You should enter a stock symbol
    """
    args_schema: Type[CompanyOverviewToolArgsSchema] = CompanyOverviewToolArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)
prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)

In [12]:
from langchain.agents import create_sql_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

llm = ChatOpenAI(temperature=0.1)

db = SQLDatabase.from_uri("sqlite:///movies.sqlite")

toolkit = SQLDatabaseToolkit(
    db=db,
    llm=llm,
)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)
agent.invoke(
    "Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue."
)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


directors, movies
Invoking: `sql_db_schema` with `movies`



CREATE TABLE movies (
	id INTEGER, 
	original_title VARCHAR, 
	budget INTEGER, 
	popularity INTEGER, 
	release_date TEXT, 
	revenue INTEGER, 
	title TEXT, 
	vote_average REAL, 
	vote_count INTEGER, 
	overview TEXT, 
	tagline TEXT, 
	uid INTEGER, 
	director_id INTEGER DEFAULT 0 NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from movies table:
id	original_title	budget	popularity	release_date	revenue	title	vote_average	vote_count	overview	tagline	uid	director_id
43597	Avatar	237000000	150	2009-12-10	2787965087	Avatar	7.2	11800	In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but 	Enter the World of Pandora.	19995	4762
43598	Pirates of the Caribbean: At World's End	300000000	139	2007-05-19	961000000	Pirates of the Caribbean: At World's End	6.9	4500	Captain Barbossa, long believed to be dead, has come back to

{'input': 'Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue.',
 'output': 'The movies that have the highest votes but the lowest budgets are:\n\n1. "Inception" directed by director_id 4765, with a revenue of $825,532,764 and a budget of $160,000,000.\n2. "The Dark Knight" directed by director_id 4765, with a revenue of $1,004,558,444 and a budget of $185,000,000.\n3. "Avatar" directed by director_id 4762, with a revenue of $2,787,965,087 and a budget of $237,000,000.\n4. "The Avengers" directed by director_id 4769, with a revenue of $1,519,557,910 and a budget of $220,000,000.\n5. "Deadpool" directed by director_id 5151, with a revenue of $783,112,979 and a budget of $58,000,000.\n6. "Interstellar" directed by director_id 4765, with a revenue of $675,120,017 and a budget of $165,000,000.\n7. "Django Unchained" directed by director_id 4927, with a revenue of $425,368,238 and a budget of $100,00